In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score
from sklearn import ensemble, linear_model, neural_network, naive_bayes
from scipy.sparse import lil_matrix, csr_matrix, save_npz, load_npz
import time
import math
import collections
import itertools as it
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [5]:
X_train = load_npz('proper_datasets/X_train.npz')
X_val = load_npz('proper_datasets/X_val.npz')

y_train = np.loadtxt('proper_datasets/y_train.csv')
y_val = np.loadtxt('proper_datasets/y_val.csv')

X_test = load_npz('proper_datasets/X_test.npz')

len(y_train), len(y_val)

(2430981, 303925)

In [5]:
def build_combinations(param_grid):
    
    '''
    Create all possible hyperparameter combinations for a given model 
    '''
  
    combinations = []

    allNames = sorted(param_grid)
    tmp_combs = list(it.product(*(param_grid[Name] for Name in allNames)))
    
    for combi in tmp_combs:
        my_dict = {}
        my_dict[allNames[0]] = combi[0]
    
        for i in range(1,len(allNames)):
            my_dict[allNames[i]] = combi[i]
        
        combinations.append(my_dict)
        
    return combinations

clf_models = collections.OrderedDict({#'knn': neighbors.KNeighborsClassifier(),
                                     #'xgb1': XGBClassifier(
                                        #learning_rate =0.1,
                                        #n_estimators=100,
                                        #max_depth=5,
                                        #min_child_weight=1,
                                        #gamma=0,
                                        #subsample=0.8,
                                        #colsample_bytree=0.8,
                                        #objective= 'binary:logistic',
                                        #nthread=4,
                                        #scale_pos_weight=1,
                                        #seed=1234,
                                        #random_state=1234),
                                     #'xgb2': XGBClassifier(
                                        #learning_rate =0.1,
                                        #n_estimators=100,
                                        #max_depth=5,
                                        #min_child_weight=1,
                                        #gamma=0,
                                        #subsample=0.8,
                                        #colsample_bytree=0.8,
                                        #objective= 'binary:logistic',
                                        #nthread=4,
                                        #scale_pos_weight=1,
                                        #seed=1234,
                                        #random_state=1234),  
                                     #'xgb3': XGBClassifier(
                                        #learning_rate =0.1,
                                     #   n_estimators=50,
                                        #max_depth=5,
                                        #min_child_weight=1,
                                        #gamma=0,
                                        #subsample=0.8,
                                        #colsample_bytree=0.8,
                                        #objective= 'binary:logistic',
                                        #nthread=4,
                                        #scale_pos_weight=1,
                                     #   seed=1234,
                                     #   random_state=1234),   
                                     #'xgb4': XGBClassifier(
                                     #   #learning_rate =0.1,
                                     #   n_estimators=50,
                                     #   #max_depth=5,
                                     #   #min_child_weight=1,
                                     #   #gamma=0,
                                     #   #subsample=0.8,
                                     #   #colsample_bytree=0.8,
                                     #   #objective= 'binary:logistic',
                                     #   #nthread=4,
                                     #   #scale_pos_weight=1,
                                     #  seed=1234,
                                     #  random_state=1234),     
                                     #'rf': ensemble.RandomForestClassifier(),
                                     #'kersvm': svm.SVC(kernel = 'rbf', probability=True), 
                                     #'linsvm': svm.SVC(kernel = 'linear', probability=True), 
                                     ###'gpc': gaussian_process.GaussianProcessClassifier(), # doesnt exist in 0.16
                                     #'gnb': naive_bayes.GaussianNB(),
                                     #'gbm': ensemble.GradientBoostingClassifier(), 
                                     'logr': linear_model.LogisticRegression(),
                                     'sgd': linear_model.SGDClassifier(),
                                     #'mlp_nn': neural_network.MLPClassifier(shuffle=True, early_stopping=True) # doesnt exist in 0.16
                                      #'et': ensemble.ExtraTreesClassifier(),
                                      })

clf_param_grid = {#'knn': {'n_neighbors': [1,3,5]}, #,7,9]}, #,11,13,15,17,19,21,23,25,27,29]},
                  #'xgb1': {'max_depth': [3, 6], 'min_child_weight': [1], 'gamma': [0, 3], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1], 'max_delta_step': [0, 5]},
                  #'xgb2': {'max_depth': [3, 6], 'min_child_weight': [1], 'gamma': [0, 3], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1], 'max_delta_step': [0, 5]},
                  #'xgb3': {'learning_rate': [0.01, 0.025, 0.05], 'max_depth': [5,10], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1]},
                  #'xgb4': {'n_estimators': [10,50,100,200], 'gamma': [0, 3], 'learning_rate': [0.01, 0.025, 0.05,0.1,0.2,0.3], 'max_depth': [5,10], 'max_delta_step': [0, 5], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1]},
                  # 'rf': {'n_estimators': [50,100,200], 'max_features': [None], 'min_samples_leaf': [1, 5]}, 
                  #'kersvm': {'C': [1/32.0, 1/8.0, 1/2.0, 2, 8, 32], 'gamma':[1/32.0, 1/8.0, 1/2.0, 2, 8, 32]},
                  #'linsvm': {'C': [1/32.0, 1/8.0, 1/2.0, 2, 8, 32]},
                  #'gpc': {'n_restarts_optimizer': [0]},
                  #'gnb': {},
                  #'gbm': {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3,6]},
                  'logr': {'solver': ['saga', 'sag'], 'C': [0.0001, 0.001, 0.01, 0.1, 1, 10 , 100, 1e10]},
                  'sgd': {'loss': ['log'], 'shuffle': [True, False], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10 , 100, 1e10], 'penalty': ['elasticnet'], 'l1_ratio': [0,0.5,1]},
                  #'mlp_nn': {'activation': ['relu', 'logistic'], 
                  #           'hidden_layer_sizes': [(5,), (10,), (20, ), (100,), (200,), 
                  #                                  (5,5), (10,10), (20, 20), (100,100), (200,200),
                  #                                  (5,5,5), (10,10,10), (20, 20,20), (100,100,100), (200,200,200)], 
                  #           'alpha': [0.0001, 0.001, 0.01, .1, 1]
                  #          } 
                  #'et': {'n_estimators': [50,100,200], 'max_features': ['auto', None], 'min_samples_leaf': [1, 5, 10]},
                  }

unique_ys = np.unique(y_train)
sgd_lower = np.linspace(0,len(y_train),100, dtype = np.int)
sgd_upper = sgd_lower[1:]
sgd_lower = sgd_lower[:-1]

for idx, (model_id, model_class) in enumerate(clf_models.items()):
    
    param_grid = clf_param_grid[model_id]
    
    #print "\n =================================== \n" \
    #      "Starting model selection for model: {}".format(model_id)

    start_time = time.time()

    if param_grid != {}:
        combinations = build_combinations(param_grid)
        
    for i in range(len(combinations)):  
    
        comb_time = time.time()
    
        combination = combinations[i]
        
        print(model_id, i, combination)
           
        # load the current parameter value into the model and perform model selection
        if combination != {}:
            model_class.set_params(**combination)
            
        #print(model_class)            

        if model_id in ['sgd', 'logr', 'gnb']:
            #for batch in range(len(sgd_lower)):
                #model_class.partial_fit(X.iloc[sgd_lower[batch]:sgd_upper[batch], :], 
                #                        y_train[sgd_lower[batch]:sgd_upper[batch]], 
                #                        classes=unique_ys)
            #model_class.partial_fit(X.iloc[:1000, :], y_train[:1000], classes=unique_ys)
            model_class.fit(X_train, y_train)
            #preds = model_class.predict(X_val.iloc[:, :])
            ##print(roc_auc_score(y_val, preds), accuracy_score(y_val, preds))
            probs = model_class.predict_proba(X_val)[:, 1]
            print(roc_auc_score(y_val, probs), math.sqrt(mean_squared_error(y_val, probs))) #, accuracy_score(y_val, probs))
            
        #np.savetxt('output/base_learners_class/r1_' + str(model_id) + '_' + str(i) + '.csv', preds, delimiter=',')
        np.savetxt('output/base_learners_probs/r1_sams_' + str(model_id) + '_' + str(i) + '.csv', probs, delimiter=',')
        
        print('Time: %.5f' % (time.time()-comb_time))

logr 0 {'C': 0.0001, 'solver': 'saga'}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.6957531538557002 0.025771153405458685
Time: 319.99630
logr 1 {'C': 0.0001, 'solver': 'sag'}
0.709588356678439 0.025771141086961834
Time: 53.79408
logr 2 {'C': 0.001, 'solver': 'saga'}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.768939906258383 0.02576332480652155
Time: 286.59739
logr 3 {'C': 0.001, 'solver': 'sag'}
0.7780029291932661 0.025764142586747637
Time: 112.74945
logr 4 {'C': 0.01, 'solver': 'saga'}


...

0.5 0.499996052405284
Time: 5.88834
sgd 46 {'alpha': 10000000000.0, 'l1_ratio': 1, 'loss': 'log', 'penalty': 'elasticnet', 'shuffle': True}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5 0.49999605240528355
Time: 8.86251
sgd 47 {'alpha': 10000000000.0, 'l1_ratio': 1, 'loss': 'log', 'penalty': 'elasticnet', 'shuffle': False}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.5 0.499996052405284
Time: 5.73633


In [ ]:
def build_combinations(param_grid):
    
    '''
    Create all possible hyperparameter combinations for a given model 
    '''
  
    combinations = []

    allNames = sorted(param_grid)
    tmp_combs = list(it.product(*(param_grid[Name] for Name in allNames)))
    
    for combi in tmp_combs:
        my_dict = {}
        my_dict[allNames[0]] = combi[0]
    
        for i in range(1,len(allNames)):
            my_dict[allNames[i]] = combi[i]
        
        combinations.append(my_dict)
        
    return combinations

clf_models = collections.OrderedDict({#'knn': neighbors.KNeighborsClassifier(),
                                     #'xgb1': XGBClassifier(
                                        #learning_rate =0.1,
                                        #n_estimators=100,
                                        #max_depth=5,
                                        #min_child_weight=1,
                                        #gamma=0,
                                        #subsample=0.8,
                                        #colsample_bytree=0.8,
                                        #objective= 'binary:logistic',
                                        #nthread=4,
                                        #scale_pos_weight=1,
                                        #seed=1234,
                                        #random_state=1234),
                                     #'xgb2': XGBClassifier(
                                        #learning_rate =0.1,
                                        #n_estimators=100,
                                        #max_depth=5,
                                        #min_child_weight=1,
                                        #gamma=0,
                                        #subsample=0.8,
                                        #colsample_bytree=0.8,
                                        #objective= 'binary:logistic',
                                        #nthread=4,
                                        #scale_pos_weight=1,
                                        #seed=1234,
                                        #random_state=1234),  
                                     #'xgb3': XGBClassifier(
                                        #learning_rate =0.1,
                                     #   n_estimators=50,
                                        #max_depth=5,
                                        #min_child_weight=1,
                                        #gamma=0,
                                        #subsample=0.8,
                                        #colsample_bytree=0.8,
                                        #objective= 'binary:logistic',
                                        #nthread=4,
                                        #scale_pos_weight=1,
                                     #   seed=1234,
                                     #   random_state=1234),   
                                     #'xgb4': XGBClassifier(
                                     #   #learning_rate =0.1,
                                     #   n_estimators=50,
                                     #   #max_depth=5,
                                     #   #min_child_weight=1,
                                     #   #gamma=0,
                                     #   #subsample=0.8,
                                     #   #colsample_bytree=0.8,
                                     #   #objective= 'binary:logistic',
                                     #   #nthread=4,
                                     #   #scale_pos_weight=1,
                                     #  seed=1234,
                                     #  random_state=1234),     
                                     #'rf': ensemble.RandomForestClassifier(),
                                     #'kersvm': svm.SVC(kernel = 'rbf', probability=True), 
                                     #'linsvm': svm.SVC(kernel = 'linear', probability=True), 
                                     ###'gpc': gaussian_process.GaussianProcessClassifier(), # doesnt exist in 0.16
                                     #'gnb': naive_bayes.GaussianNB(),
                                     #'gbm': ensemble.GradientBoostingClassifier(),
                                     #'logr': linear_model.LogisticRegression(),
                                     #'sgd': linear_model.SGDClassifier(loss='log', shuffle =False),
                                     'mlp_nn': neural_network.MLPClassifier(shuffle=True, early_stopping=True) # doesnt exist in 0.16
                                      #'et': ensemble.ExtraTreesClassifier(),
                                      })

clf_param_grid = {#'knn': {'n_neighbors': [1,3,5]}, #,7,9]}, #,11,13,15,17,19,21,23,25,27,29]},
                  #'xgb1': {'max_depth': [3, 6], 'min_child_weight': [1], 'gamma': [0, 3], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1], 'max_delta_step': [0, 5]},
                  #'xgb2': {'max_depth': [3, 6], 'min_child_weight': [1], 'gamma': [0, 3], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1], 'max_delta_step': [0, 5]},
                  #'xgb3': {'learning_rate': [0.01, 0.025, 0.05], 'max_depth': [5,10], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1]},
                  #'xgb4': {'n_estimators': [10,50,100,200], 'gamma': [0, 3], 'learning_rate': [0.01, 0.025, 0.05,0.1,0.2,0.3], 'max_depth': [5,10], 'max_delta_step': [0, 5], 'scale_pos_weight': [y_train[y_train==1].count()/y_train[y_train==0].count(), 1]},
                  # 'rf': {'n_estimators': [50,100,200], 'max_features': [None], 'min_samples_leaf': [1, 5]}, 
                  #'kersvm': {'C': [1/32.0, 1/8.0, 1/2.0, 2, 8, 32], 'gamma':[1/32.0, 1/8.0, 1/2.0, 2, 8, 32]},
                  #'linsvm': {'C': [1/32.0, 1/8.0, 1/2.0, 2, 8, 32]},
                  #'gpc': {'n_restarts_optimizer': [0]},
                  #'gnb': {},
                  #'gbm': {'n_estimators': [100, 300, 500], 'learning_rate': [0.01, 0.1, 0.5], 'max_depth': [3,6]},
                  #'logr': {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10 , 100, 1e10]},
                  #'sgd': {'loss': ['log', 'modified_huber'], 'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10 , 100, 1e10], 'penalty': ['elasticnet'], 'l1_ratio': [0,0.5,1]},
                  'mlp_nn': {'activation': ['logistic', 'relu'], 
                             'hidden_layer_sizes': [(5,), (10,), (20, ), (100,), (200,), 
                                                    (5,5), (10,10), (20, 20), (100,100), (200,200),
                                                    (5,5,5), (10,10,10), (20, 20,20), (100,100,100), (200,200,200)], 
                             'alpha': [0.0001, 0.001, 0.01, 0.1, 1]
                            } 
                  #'et': {'n_estimators': [50,100,200], 'max_features': ['auto', None], 'min_samples_leaf': [1, 5, 10]},
                  }

unique_ys = np.unique(y_train)
sgd_lower = np.linspace(0,len(y_train),100, dtype = np.int)
sgd_upper = sgd_lower[1:]
sgd_lower = sgd_lower[:-1]

for idx, (model_id, model_class) in enumerate(clf_models.items()):
    
    param_grid = clf_param_grid[model_id]
    
    #print "\n =================================== \n" \
    #      "Starting model selection for model: {}".format(model_id)

    start_time = time.time()
    
    if param_grid != {}:
        combinations = build_combinations(param_grid)
        
    for i in range(len(combinations)):  
    
        comb_time = time.time()
        
        combination = combinations[i]
        
        print(model_id, i, combination)
    
        # load the current parameter value into the model and perform model selection
        if combination != {}:
            model_class.set_params(**combination)
            
        if model_id == 'mlp_nn':
            #model_class.fit(X, y_train)  
            #for batch in range(len(sgd_lower)):
                #model_class.partial_fit(X.iloc[sgd_lower[batch]:sgd_upper[batch], :], 
                #                        y_train[sgd_lower[batch]:sgd_upper[batch]], 
                #                        classes=unique_ys)
            #model_class.partial_fit(X.iloc[:1000, :], y_train[:1000], classes=unique_ys)
            model_class.fit(X_train, y_train)
            #preds = model_class.predict(X_val.iloc[:, :])
            ##print(roc_auc_score(y_val, preds), accuracy_score(y_val, preds))
            probs = model_class.predict_proba(X_val)[:, 1]
            print(roc_auc_score(y_val, probs), math.sqrt(mean_squared_error(y_val, probs))) #, accuracy_score(y_val, probs))
            
            test_probs = model_class.predict_proba(X_test)[:, 1]
                        
        else:
            continue
            
        #np.savetxt('output/base_learners_class/r2_' + str(model_id) + '_' + str(i) + '.csv', preds, delimiter=',')
        np.savetxt('output/base_learners_probs/r2_sams_' + str(model_id) + '_' + str(i) + '.csv', probs, delimiter=',')
        np.savetxt('output/base_learners_test_probs/r2_sams_' + str(model_id) + '_' + str(i) + '.csv', test_probs, delimiter=',')
        
        print('Time: %.5f' % (time.time()-comb_time))